C# supports parallel execution of code through multithreading. A thread is an independent execution path, able to run simultaneously with other threads.  

 * A thread ends when the delegate passed to the Thread’s constructor finishes executing.  
 * Once ended, a thread cannot restart.  
 * The CLR assigns each thread its own memory stack so that local variables are kept separate  
 * Data sharing between threads, with instance methods and static methods.
 * A thread, while blocked, doesn't consume CPU resources.  

Thread Synchronization
* Thread safety : Protection from indeterminacy in a multithreading context is called thread safety.
* Exclusive Lock


Multithreading is managed internally by a thread scheduler,      
A thread scheduler ensures all active threads are allocated appropriate execution time   

**Time Slicing** on single processor computer  

On a single-processor computer, a thread scheduler performs time-slicing  
a time-slice is typically in the tens-of-milliseconds region     
switching context between one thread and another few-microseconds region    

**Genuine Concurrency**  
On a multi-processor computer, multithreading is implemented with a mixture of time-slicing and genuine concurrency  

**Threads vs Processes**
* Threads run inside a process, A process is managed by operating system.
* Threading can share heap memoery this makes them usefull, One thread can fetch data in background while other thread display data on screen.  


**Multithreading uses**
* Maintaining a responsive user interface
* Making efficient use of an otherwise blocked CPU 
* Parallel programming
* Speculative execution
* Allowing requests to be processed simultaneously


```
lock () {}
```


### Vocabulary
* Critical Section
* Thraed Safety
* multithreading context
* nondeterministic ouput
* Blocks: When one thread block all other remaining from accessing critical section.
* Locker: When two threads simultaneously contend a lock
* Thread scheduler 
* Execution Time
* Time slicing : rapidly switching execution between each of the active threads. 
* Genuine Concurrency: multithreading on a multi processor
* preempted : When its execution is interrupted due to an external factor such as time-slicing.


### Tips
* Shared data is the primary cause of complexity and obscure errors in multithreading.
* A good strategy is to encapsulate multithreading logic into reusable classes that can be independently examined and tested. The Framework itself offers many higher-level threading constructs.


In [2]:
// Notebook usings
using System;
using System.Threading;

In [2]:
/*
 multithreaded main
 IsAlive property.
*/

class ThreadTest
{
  public static void Main()
  {
    //  A thread ends when the delegate passed to the Thread’s constructor finishes executing.
    Thread t = new Thread (WriteY);          // Kick off a new thread
    t.Start();                               // running WriteY()
 
    // Simultaneously, do something on the main thread.
    for (int i = 0; i < 10; i++) Console.Write ("x");
  }
 
  // The CLR assigns each thread its own memory stack so that local variables are kept separate
  // worker thread
  static void WriteY()
  {
    for (int i = 0; i < 10; i++) Console.Write ("y");
  }
}

// notebook invokation

ThreadTest.Main();

// output : xxxxxxxxxxyyyyyyyyyy

xxxxxxxxxxyyyyyyyyyy

In [1]:
static void Main() 
{
  new Thread (Go).Start();      // Call Go() on a new thread
  Go();                         // Call Go() on the main thread
}
 
// The CLR assigns each thread its own memory stack so that local variables are kept separate
static void Go()
{
  // Declare and use a local variable - 'cycles'
  for (int cycles = 0; cycles < 5; cycles++) Console.Write ('?');
}

// Notebook Invocation
Main();

// output: ?????????
// note that output, has 5 ? from childthread & 5 ? from main thread, The cycles as it's own scope inside the thread.
// each thread local var are stored in "thread's memoery stack."

Error: (3,7): error CS0246: The type or namespace name 'Thread' could not be found (are you missing a using directive or an assembly reference?)

In [5]:
// Threads share data if they have a common reference to the same object instance. For example:

// static methods vs instance methods.
class ThreadTest
{
  bool done;
 
  public static void Main()
  {
    // instance the thread test class.
    ThreadTest tt = new ThreadTest();   // Create a common instance
    // start the instance object method, & pass the instance method.
    new Thread (tt.Go).Start();
    tt.Go();
  }
 
  // Note that Go is now an instance method
  void Go() 
  {
     if (!done) { done = true; Console.WriteLine ("Done"); }
  }
}

// notebook invocation
ThreadTest.Main();

// output: Done
//  common reference data is being shared between threads if the method is instance of an object.

Done


In [7]:
// Static fields offer another way to share data between threads.

class ThreadTest 
{
  static bool done;    // Static fields are shared between all threads
 
  public static void Main()
  {
    // new Thread (Go).Start();
    // Go();

    new Thread (Go2).Start();
    Go2();
  }
 
  static void Go()
  {
    if (!done) { done = true; Console.WriteLine("Done"); }
  }


  // thread safety
  static void Go2()
  {
    if (!done) { Console.WriteLine("Done"); done = true; }
  }
}

// notebook invocation
ThreadTest.Main();

// output : Done
// static data is shared between static methods invoked in threading.

// Go2() Without thread safety.
// on Go2()
// output : Done
// Done

Done
Done


In [11]:
// exclusive lock
// The remedy is to obtain an exclusive lock while reading and writing to the common field  
class ThreadSafe 
{
  static bool done;
  static readonly object locker = new object();
 
 public static void Main()
  {
    new Thread (Go).Start();
    Go();
  }
 
  static void Go()
  {
    // Code that's protected in such a manner — from indeterminacy in a multithreading context — is called thread-safe.  
    lock (locker)
    {
      if (!done) { Console.WriteLine ("Done"); done = true; }
    }
  }
}

// notebook invocation
ThreadSafe.Main();

// output : Done

Done


In [ ]:
// You can wait for another thread to end by calling its Join method

static void Main()
{
  Thread t = new Thread (Go);
  // While waiting on a Sleep or Join, a thread is blocked and so does not consume CPU resources.
  t.Start();
  //  join blocks the thread.
  t.Join();
  Console.WriteLine ("Thread t has ended!");
}
 
static void Go()
{
  for (int i = 0; i < 1000; i++) Console.Write ("y");
}

Main();

// pauses the current thread.
//  sleep blocks the thread.
Thread.Sleep (TimeSpan.FromHours (1));  // sleep for 1 hour
Thread.Sleep (500);                     // sleep for 500 milliseconds

###  Creating and Starting Threads

In [ ]:
public delegate void ThreadStart();

class ThreadTest
{
  static void Main() 
  {
    Thread t = new Thread (new ThreadStart (Go));
 
    t.Start();   // Run Go() on the new thread.
    Go();        // Simultaneously run Go() in the main thread.
  }
 
  static void Go()
  {
    Console.WriteLine ("hello!");
  }
}

In [ ]:
// allowing C# to infer the ThreadStart delegate:

Thread t = new Thread (Go);    // No need to explicitly use ThreadStart

// another way with, lambda expression or anonymous method.
static void Main()
{
  Thread t = new Thread ( () => Console.WriteLine ("Hello!") );
  t.Start();
}

#### Passing Data to a Thread

In [ ]:
// The easy way.

static void Main()
{
    // The easiest way to pass arguments to a thread’s target method is to execute a lambda expression that calls the method with the desired arguments
  Thread t = new Thread ( () => Print ("Hello from t!") );
  t.Start();
}
 
static void Print (string message) 
{
  Console.WriteLine (message);
}

In [2]:
// multi-statement lambda
new Thread (() =>
{
  Console.WriteLine ("I'm running on another thread!");
  Console.WriteLine ("This is so easy!");
}).Start();

I'm running on another thread!
This is so easy!


In [ ]:
new Thread (delegate()
{
  ...
}).Start();

In [3]:
static void Main()
{
  Thread t = new Thread (Print);
  t.Start ("Hello from t!");
}
 
static void Print (object messageObj)
{
  string message = (string) messageObj;   // We need to cast here
  Console.WriteLine (message);
}

In [4]:
//  Thread’s constructor is overloaded to accept either of two delegates
public delegate void ThreadStart();

// Note: it can only accept object type, So we need to cast to process the params passed to it.
public delegate void ParameterizedThreadStart (object obj);

In [5]:
for (int i = 0; i < 10; i++)
  new Thread (() => Console.Write (i)).Start();

  //ouput: 11344678910 <- nondeterministic output.


  // fixing above code.
  for (int i = 0; i < 10; i++)
{
  // Variable temp is now local to each loop iteration. Therefore, each thread captures a different memory location 
  int temp = i;
  new Thread (() => Console.Write (temp)).Start();
}

11344678910

In [6]:
// problem with nondeterministic output, with for and foreach.
string text = "t1";
Thread t1 = new Thread ( () => Console.WriteLine (text) );
 
text = "t2";
Thread t2 = new Thread ( () => Console.WriteLine (text) );
 
t1.Start();
t2.Start();

// output : 
//t2
//t2

t2
t2


#### Naming Threads

In [7]:
class ThreadNaming
{
  static void Main()
  {
    Thread.CurrentThread.Name = "main";
    Thread worker = new Thread (Go);
    worker.Name = "worker";
    worker.Start();
    Go();
  }
 
  static void Go()
  {
    Console.WriteLine ("Hello from " + Thread.CurrentThread.Name);
  }
}

#### Foreground and Background Threads

* By default, threads you create explicitly are foreground threads. 
* Foreground threads keep the application alive 

In [ ]:
class PriorityTest
{
  static void Main (string[] args)
  {
    Thread worker = new Thread ( () => Console.ReadLine() );
    if (args.Length > 0) worker.IsBackground = true;
    worker.Start();
  }
}